In [31]:
names = open("names.txt").read().splitlines()
names[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

# Lets do name predition using probablity

In [32]:
# Exercises:
# E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?
# E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?
# E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?
# E04: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?
# E05: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?
# E06: meta-exercise! Think of a fun/interesting exercise and complete it.

In [33]:
# will start with trigram model using probabality
# given two words i need to egt th ecount of 3rd one
# here i will be using counter instead of dictionary
from collections import Counter
counter = Counter()
for name in names:
    name_modified = "."+name+"."
    counter.update(["".join((char, char_next1, char_next2)) for char, char_next1, char_next2 in zip(name_modified, name_modified[1:], name_modified[2:])])


# i wanted to plot this in 3d graph, , it looks a lot of effor to show it meaningfully.
# for now i will try to work, without graph
# so after getitn the count i thin ki need to fit it into 


In [34]:
import torch

N = torch.zeros((27,27,27), dtype=torch.float32)

# let us assign the the alphabets to integers
char_set = set()
for name in names:
    modified_name = "."+name+"."
    
    char_set.update(modified_name)

i2c = {}
c2i = {}
char_set = sorted(char_set)
for i, item in enumerate(char_set):
    i2c.update({i:item})
    c2i.update({item:i})

N


tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [35]:
# Now we have the probablity
# we dont have all combination of bigram avialable in corpus, 
# so in some casses all trigrams occurane might be zeor
# hence there probablity will aalso be zero, for such cassed we need to give a default probabality of 1/27
# which is euqal probabaliry of accurance for all characters
# so we might zero probabalities also
#default probabality = 1/27 since we have 27 characters
P = torch.full((27,27,27),1/27, dtype=torch.float32)
for item in counter.items():
    x, y, z = item[0]
    # print(x,y,z)
    print(item[1])
    N[c2i[x],c2i[y],c2i[z]] = item[1]

for item in counter.items():
    x, y, z = item[0]
    P[c2i[x],c2i[y],c2i[z]] = N[c2i[x],c2i[y],c2i[z]] / N[c2i[x],c2i[y]].sum()

# TODO
# do it using tensor operation of sum
N
        

288
100
72
174
104
69
54
78
147
903
243
161
93
124
142
76
173
201
822
337
684
152
21
37
61
81
352
236
329
287
44
14
34
121
175
393
95
384
226
188
537
518
505
8
5
77
683
154
142
76
353
976
953
190
76
15
35
18
259
119
160
259
203
270
488
81
93
40
61
114
178
279
166
160
84
201
18
19
20
628
92
355
482
950
499
27
14
193
146
91
203
102
161
26
21
22
110
220
1453
136
190
42
68
341
1503
162
25
61
1673
165
120
103
129
234
22
24
17
28
92
31
151
145
83
11
33
26
463
234
483
381
201
62
197
443
602
72
19
14
156
101
202
627
269
133
219
173
196
98
148
469
825
633
185
1714
211
271
790
1509
366
94
51
152
35
34
189
163
285
858
135
154
63
470
101
111
153
575
366
92
42
60
83
727
89
21
83
106
108
12
18
595
35
237
49
23
113
646
69
38
27
11
41
253
68
60
177
189
122
231
55
44
104
153
228
69
6
38
175
31
145
63
138
101
80
623
103
42
505
241
607
52
90
110
86
93
57
21
287
180
601
252
240
261
23
31
128
136
431
274
177
44
168
284
153
389
716
91
56
58
27
25
250
134
58
63
10
6
74
124
207
46
117
669
511
383
279
83
371
2

tensor([[[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0., 207., 190.,  ...,  27., 173., 152.],
         [  0., 169.,   0.,  ...,   0.,   4.,   0.],
         ...,
         [  0.,  57.,   0.,  ...,   1.,  17.,  11.],
         [  0., 246.,   0.,  ...,   0.,   0.,   2.],
         [  0., 456.,   0.,  ...,   0.,  91.,   1.]],

        [[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [ 40.,   0.,   5.,  ...,   0.,  20.,  11.],
         [ 36.,  28.,  20.,  ...,   0.,  12.,   0.],
         ...,
         [ 11.,   5.,   0.,  ...,  17.,   6.,   3.],
         [163., 389.,  13.,  ...,   0.,  16.,  40.],
         [ 38., 123.,   0.,  ...,   0.,  12.,  22.]],

        [[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [ 46.,   5.,   5.,  ...,   4.,  31.,   4.],
         [  1.,   8.,   0.,  ...,   0.,   9.,   0.],
         ...,
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [ 55.,   4.,   1.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.]],

In [36]:
# now that i have probablities i will do sampling
g = torch.Generator().manual_seed(873269874)
sample_char = torch.multinomial(P,num_samples = 1, replacement = True, generator = g)

sample_char

RuntimeError: prob_dist must be 1 or 2 dim

In [ ]:
g = torch.Generator().manual_seed(873269874)
first_char = "a"
idx1 = c2i[first_char]
second_char = "b"
idx2 = c2i[second_char]

In [ ]:
P[2,0]

tensor([0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370,
        0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370,
        0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370])

In [ ]:
# lets do sampling manualy first
# to start assume with any two characters lets say a and b

for i in range(10):
    gen_name_list = []
    while True:
        # print(idx1, idx2)
        sample_char_idx = torch.multinomial(P[idx1,idx2],num_samples = 1, replacement = True, generator = g)
        idx1=idx2
        idx2=sample_char_idx.item()
        sample_char = i2c[sample_char_idx.item()]
        # print(sample_char)
        gen_name_list.append(sample_char)
        if sample_char_idx ==0:
            break
    gen_name = "".join(gen_name_list)
    print(gen_name)


fgswaelairenfor.
carius.
rayon.
wliya.
quci.
ch.
wa.
emali.
lnrvqvonfii.
borariel.


In [ ]:
# now that we have the names
# let us calculayte the loss
# loss is given by how far is the prediction form reality
# probabality is one way to do this
# so to get the loss we need to get the probablity of each occurance
# and multiply those
# so we will get the occurance 

In [ ]:
log_likelyhood = 0
n = 0
for name in names[:5]:
    name_modified= "."+name+"."
    for ch1, ch2, ch3 in zip(name_modified[:],name_modified[1:], name_modified[2:]):
        # idx1, idx2,idx3 =i2c[ch1], i2c[ch2], i2c[ch3]
        prob = P[c2i[ch1], c2i[ch2], c2i[ch3]]
        logprob = torch.log(prob)
        log_likelyhood +=logprob 
        n+=1

log_likelyhood = -log_likelyhood
nll = log_likelyhood/n
nll


tensor(1.8257)

# Now we will do same using neural netwok

In [ ]:
# start with creating features and labels
xs = []
ys = []
for name in names:
    name_modified = "."+name+"."
    for ch1, ch2, ch3 in zip(name_modified,name_modified[1:], name_modified[2:]):
        xs.append([c2i[ch1],c2i[ch2]])
        ys.append(c2i[ch3])

print(xs)
print(ys)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
len(xs)


[[0, 5], [5, 13], [13, 13], [13, 1], [0, 15], [15, 12], [12, 9], [9, 22], [22, 9], [9, 1], [0, 1], [1, 22], [22, 1], [0, 9], [9, 19], [19, 1], [1, 2], [2, 5], [5, 12], [12, 12], [12, 1], [0, 19], [19, 15], [15, 16], [16, 8], [8, 9], [9, 1], [0, 3], [3, 8], [8, 1], [1, 18], [18, 12], [12, 15], [15, 20], [20, 20], [20, 5], [0, 13], [13, 9], [9, 1], [0, 1], [1, 13], [13, 5], [5, 12], [12, 9], [9, 1], [0, 8], [8, 1], [1, 18], [18, 16], [16, 5], [5, 18], [0, 5], [5, 22], [22, 5], [5, 12], [12, 25], [25, 14], [0, 1], [1, 2], [2, 9], [9, 7], [7, 1], [1, 9], [9, 12], [0, 5], [5, 13], [13, 9], [9, 12], [12, 25], [0, 5], [5, 12], [12, 9], [9, 26], [26, 1], [1, 2], [2, 5], [5, 20], [20, 8], [0, 13], [13, 9], [9, 12], [12, 1], [0, 5], [5, 12], [12, 12], [12, 1], [0, 1], [1, 22], [22, 5], [5, 18], [18, 25], [0, 19], [19, 15], [15, 6], [6, 9], [9, 1], [0, 3], [3, 1], [1, 13], [13, 9], [9, 12], [12, 1], [0, 1], [1, 18], [18, 9], [9, 1], [0, 19], [19, 3], [3, 1], [1, 18], [18, 12], [12, 5], [5, 20], [

196113

In [ ]:
# we have training data
# now we need weight then we will do X*W which will repesent the log of count
# we will take exponential of to make is reperentation of count
# but this exponential is a like softmax
# then we calculate the loss 
# then we do back propagation



In [72]:
import torch.nn.functional as F

xenc = F.one_hot(xs, num_classes=27).float()
xenc = xenc.flatten(start_dim = 1)
xenc.shape


torch.Size([196113, 54])

In [75]:
g= torch.Generator().manual_seed(1234567890)
W  = torch.randn((54,27), generator = g, requires_grad = True)

In [77]:
# since we have input of 54 and the output possiblity dimention is 27  hence 54*27
n = len(xs)
print(n)
for epoch in range(500):
    logits = xenc @ W
    #logits are log of count
    logit_exp = logits.exp()
    probs = logit_exp/logit_exp.sum(1, keepdims = True)
    probs
    loss = -probs[torch.arange(n),ys].log().mean()

    print(loss)
    W.grad =None
    loss.backward()
    W.data += -50*W.grad

196113
tensor(2.2621, grad_fn=<NegBackward0>)


tensor(2.2618, grad_fn=<NegBackward0>)
tensor(2.2616, grad_fn=<NegBackward0>)
tensor(2.2613, grad_fn=<NegBackward0>)
tensor(2.2610, grad_fn=<NegBackward0>)
tensor(2.2607, grad_fn=<NegBackward0>)
tensor(2.2605, grad_fn=<NegBackward0>)
tensor(2.2602, grad_fn=<NegBackward0>)
tensor(2.2600, grad_fn=<NegBackward0>)
tensor(2.2597, grad_fn=<NegBackward0>)
tensor(2.2595, grad_fn=<NegBackward0>)
tensor(2.2593, grad_fn=<NegBackward0>)
tensor(2.2590, grad_fn=<NegBackward0>)
tensor(2.2588, grad_fn=<NegBackward0>)
tensor(2.2586, grad_fn=<NegBackward0>)
tensor(2.2584, grad_fn=<NegBackward0>)
tensor(2.2581, grad_fn=<NegBackward0>)
tensor(2.2579, grad_fn=<NegBackward0>)
tensor(2.2577, grad_fn=<NegBackward0>)
tensor(2.2575, grad_fn=<NegBackward0>)
tensor(2.2573, grad_fn=<NegBackward0>)
tensor(2.2571, grad_fn=<NegBackward0>)
tensor(2.2569, grad_fn=<NegBackward0>)
tensor(2.2567, grad_fn=<NegBackward0>)
tensor(2.2566, grad_fn=<NegBackward0>)
tensor(2.2564, grad_fn=<NegBackward0>)
tensor(2.2562, grad_fn=<N

In [88]:
#sampling names
char1 = "a"
char2 = "b"
name_list = []
for i in range(100):
    name_list = []
    while True:

        char1_idx = c2i[char1]
        char2_idx = c2i[char2]
        intial_tensor = torch.tensor([[char1_idx, char2_idx]])
        xenc = F.one_hot(intial_tensor, num_classes=27).float()
        xenc = xenc.flatten(start_dim = 1)
        logits = xenc @ W
        logit_exp = logits.exp()
        probs = logit_exp/logit_exp.sum(1, keepdims = True)
        sample_char_idx = torch.multinomial(probs, num_samples=1, replacement =True, generator = g).item()
        char = i2c[sample_char_idx]
        char1= char2
        char2 = char
        name_list.append(char)
        # print(char)
        if char==".":
            break

    name= "".join(name_list)
    print(name)

elsha.
madnn.
xaadryy.
aurie.
.
ra.
.
hura.
budarrysanidaltynyushiey.
na.
askosharyden.
kayloni.
naeleitrea.
kri.
ani.
ashin.
von.
ulah.
kyi.
suriem.
klissiastely.
urrakinoa.
vah.
wenty.
aona.
jaton.
ah.
.
bria.
lle.
.
emeys.
ph.
arishedrie.
nh.
aunatar.
dmaruwrit.
jaxsevt.
lenneryen.
rae.
.
am.
kaysh.
ardin.
camae.
ardema.
vilen.
edonesina.
khimh.
capeantikley.
ah.
brey.
er.
el.
jon.
wi.
ahammiy.
rmariaslie.
.
avia.
vonele.
kerhyre.
.
byahael.
ilattsta.
vano.
kh.
vaed.
oubrya.
ka.
konambelyah.
jola.
ja.
winianielidessoh.
emcayana.
abannzialani.
kyinaiasieni.
ai.
marickerrix.
zui.
.
usial.
amidarmen.
ayrna.
basikee.
gra.
aavon.
.
berev.
ole.
makela.
.
on.
eatan.
evge.
kydin.
um.
kedarphal.
javinirolah.
ahneigreenn.
